# [Deep Learning](https://www.cc.gatech.edu/~hays/compvision/proj6/)

## Setup

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import os.path as osp
import matplotlib.pyplot as plt
from utils import *
import student_code as sc
from torchvision.models import alexnet

data_path = osp.join('../data', '15SceneData')
num_classes = 15

# If you have a good Nvidia GPU with an appropriate environment, 
# try setting the use_GPU flag to True (the environment provided does
# not support GPUs and we will not provide any support for GPU
# computation in this project). Please note that 
# we will evaluate your implementations only using CPU mode so even if
# you use a GPU, make sure your code runs in the CPU mode with the
# environment we provided. 
use_GPU = False
if use_GPU:
    from utils_gpu import *

To train a network in PyTorch, we need 4 components:
1. **Dataset** - an object which can load the data and labels given an index.
2. **Model** - an object that contains the network architecture definition.
3. **Loss function** - a function that measures how far the network output is from the ground truth label.
4. **Optimizer** - an object that optimizes the network parameters to reduce the loss value.

This project has two main parts. In Part 1, you will train a deep network from scratch. In Part 2, you will "fine-tune" a trained network. 

## Part 0. Warm up! Training a Deep Network from Scratch

In [2]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

You do not need to code anything for this part. You will simply run the code we provided, but we want you to report the result you got. This section will also familiarize you with the steps of training a deep network from scratch. 

In [3]:
# Training parameters.
input_size = (64, 64)
RGB = False  
base_lr = 1e-2  # may try a smaller lr if not using batch norm
weight_decay = 5e-4
momentum = 0.9

We will first create our datasets, by calling the create_datasets function from student_code. This function returns a separate dataset loader for each split of the dataset (training and testing/validation). Each dataloader is used to load the datasets after appling some pre-processing transforms. In Part 1, you will be asked to add a few more pre-processing transforms to the dataloaders by modifying this function.

In [4]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Computing pixel mean and stdev...
Batch 0 / 30
Batch 20 / 30
Done, mean = 
[0.45579668]
std = 
[0.23624939]
Computing pixel mean and stdev...
Batch 0 / 60
Batch 20 / 60
Batch 40 / 60
Done, mean = 
[0.45517009]
std = 
[0.2350788]


Now we will create our network model using the SimpleNet class from student_code. The implementation provided in the SimpleNet class gives you a basic network. In Part 1, you will be asked to add a few more layers to this network. 

In [5]:
# Create the network model.
model = sc.SimpleNet(num_classes=num_classes, rgb=False, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)

SimpleNet(
  (features): Sequential(
    (0): Conv2d(1, 10, kernel_size=(9, 9), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (classifier): Conv2d(10, 15, kernel_size=(8, 8), stride=(1, 1))
)


Next we will create the loss function and the optimizer. 

In [6]:
# Create the loss function.
# see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
loss_function = nn.CrossEntropyLoss()

In [7]:
# Create the optimizer and a learning rate scheduler
optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
# Currently a simple step scheduler.
# See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
# and how to use them
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

Finally we are ready to train our network! We will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [8]:
# train the network!
params = {'n_epochs': 100, 'batch_size': 50, 'experiment': 'part1'}
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

---------------------------------------
Experiment: part1
checkpoint_file: None
n_epochs: 100
print_freq: 100
batch_size: 50
num_workers: 4
val_freq: 1
shuffle: True
experiment: part1
do_val: True
resume_optim: True
---------------------------------------
part1 Epoch 0 / 100
train part1: batch 0/29, loss 2.709, top-1 accuracy 8.000, top-5 accuracy 28.000
train part1: loss 2.708517
val part1: batch 0/59, loss 2.720, top-1 accuracy 0.000, top-5 accuracy 0.000
val part1: loss 2.708254
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 1 / 100
train part1: batch 0/29, loss 2.706, top-1 accuracy 0.000, top-5 accuracy 42.000
train part1: loss 2.706191
val part1: batch 0/59, loss 2.649, top-1 accuracy 28.000, top-5 accuracy 100.000
val part1: loss 2.709384
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 2 / 100
train part1: batch 0/29, loss 2.703, top-1 accuracy 8.000, top-5 accuracy 34.000
train part1: loss 2.699870
val part1: batch 0/59, loss 2.666, top-1 accuracy 0.000, top-5 ac

train part1: batch 0/29, loss 1.284, top-1 accuracy 48.000, top-5 accuracy 98.000
train part1: loss 1.322582
val part1: batch 0/59, loss 2.934, top-1 accuracy 16.000, top-5 accuracy 60.000
val part1: loss 2.376149
Checkpoint saved
part1 Epoch 31 / 100
train part1: batch 0/29, loss 0.906, top-1 accuracy 74.000, top-5 accuracy 96.000
train part1: loss 1.308420
val part1: batch 0/59, loss 2.465, top-1 accuracy 30.000, top-5 accuracy 76.000
val part1: loss 2.397908
Checkpoint saved
part1 Epoch 32 / 100
train part1: batch 0/29, loss 1.117, top-1 accuracy 64.000, top-5 accuracy 96.000
train part1: loss 1.288347
val part1: batch 0/59, loss 2.521, top-1 accuracy 30.000, top-5 accuracy 68.000
val part1: loss 2.329163
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 33 / 100
train part1: batch 0/29, loss 1.301, top-1 accuracy 56.000, top-5 accuracy 90.000
train part1: loss 1.198376
val part1: batch 0/59, loss 3.230, top-1 accuracy 14.000, top-5 accuracy 60.000
val part1: loss 2.352172
Chec

train part1: batch 0/29, loss 0.496, top-1 accuracy 90.000, top-5 accuracy 100.000
train part1: loss 0.501907
val part1: batch 0/59, loss 4.259, top-1 accuracy 18.000, top-5 accuracy 52.000
val part1: loss 2.810386
Checkpoint saved
part1 Epoch 63 / 100
train part1: batch 0/29, loss 0.453, top-1 accuracy 92.000, top-5 accuracy 100.000
train part1: loss 0.499241
val part1: batch 0/59, loss 4.410, top-1 accuracy 18.000, top-5 accuracy 52.000
val part1: loss 2.801498
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 64 / 100
train part1: batch 0/29, loss 0.515, top-1 accuracy 84.000, top-5 accuracy 100.000
train part1: loss 0.490560
val part1: batch 0/59, loss 4.311, top-1 accuracy 18.000, top-5 accuracy 50.000
val part1: loss 2.818260
Checkpoint saved
part1 Epoch 65 / 100
train part1: batch 0/29, loss 0.429, top-1 accuracy 86.000, top-5 accuracy 98.000
train part1: loss 0.487638
val part1: batch 0/59, loss 4.499, top-1 accuracy 16.000, top-5 accuracy 50.000
val part1: loss 2.794961
C

train part1: loss 0.427599
val part1: batch 0/59, loss 4.704, top-1 accuracy 18.000, top-5 accuracy 52.000
val part1: loss 2.912790
Checkpoint saved
part1 Epoch 95 / 100
train part1: batch 0/29, loss 0.512, top-1 accuracy 90.000, top-5 accuracy 100.000
train part1: loss 0.426167
val part1: batch 0/59, loss 4.644, top-1 accuracy 18.000, top-5 accuracy 54.000
val part1: loss 2.913283
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 96 / 100
train part1: batch 0/29, loss 0.429, top-1 accuracy 92.000, top-5 accuracy 98.000
train part1: loss 0.422736
val part1: batch 0/59, loss 4.196, top-1 accuracy 18.000, top-5 accuracy 58.000
val part1: loss 2.934891
Checkpoint saved
part1 Epoch 97 / 100
train part1: batch 0/29, loss 0.473, top-1 accuracy 90.000, top-5 accuracy 98.000
train part1: loss 0.423020
val part1: batch 0/59, loss 4.575, top-1 accuracy 18.000, top-5 accuracy 52.000
val part1: loss 2.920759
Checkpoint saved
part1 Epoch 98 / 100
train part1: batch 0/29, loss 0.600, top-1 accu

Connection is already closed.


Expect this code to take around 5 minutes on CPU or 3 minutes on GPU. Now you are ready to actually modify the functions we used to train our model. Before you move on, make sure to record the accuracy of your network from Part 0, and report it in your write up. 

## Part 1: Modifying the Dataloaders and the Simple Network create_datasets

In [ ]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

Now you will modify the create_datasets function from student_code. You will add random left-right mirroring and normalization to the transformations applied to the training dataset. You will also add normalization to the transformations applied to the testing dataset. 

In [ ]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Now you will modify SimpleNet by adding droppout, batch normalization, and additional convolution/maxpool/relu layers. You should achieve an accuracy of at least **50%**. Make sure your network passes this threshold--it is required for full credit on this section!

You can also use the following two blocks to determine the stucture of your network.

In [ ]:
# create the network model
model = sc.SimpleNet(num_classes=num_classes, rgb=False, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)

In [ ]:
# Use this block to determine the kernel size of the conv2d layer in the classifier
# first, set the kernel size of that conv2d layer to 1, and run this block
# then, use that size of input to the classifier printed by this block to
# go back and update the kernel size of the conv2d layer in the classifier
# Finally, run this block again and verify that the network output size is a scalar
# Don't forget to re-run the block above every time you update the SimpleNet class!
from torch.autograd import Variable
data, _ = train_dataset[0]
s = data.size()
data = Variable(data.view(1, *s))
if use_GPU:
    data = data.cuda()
out = model(data)
print('Network output size is ', out.size())

Next we will create the loss function and the optimizer. You do not have to modify the custom_part1_trainer in student_code if you use the same loss_function, optimizer, scheduler and parameters (n_epoch, batch_size etc.) as provided in this notebook to hit the required threshold of 50% accuracy. If you changed any of these values, it is important that you modify this function in student_code since we will not be using the notebook you submit to evaluate. 

In [ ]:
# Set up the trainer. You can modify custom_part1_trainer in
# student_copy.py if you want to try different learning settings.
custom_part1_trainer = sc.custom_part1_trainer(model)

if custom_part1_trainer is None:
    # Create the loss function.
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Create the optimizer and a learning rate scheduler.
    optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
    # Currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

    params = {'n_epochs': 100, 'batch_size': 50, 'experiment': 'part1'}
    
else:
    if 'loss_function' in custom_part1_trainer:
        loss_function = custom_part1_trainer['loss_function']
    if 'optimizer' in custom_part1_trainer:
        optimizer = custom_part1_trainer['optimizer']
    if 'lr_scheduler' in custom_part1_trainer:
        lr_scheduler = custom_part1_trainer['lr_scheduler']
    if 'params' in custom_part1_trainer:
        params = custom_part1_trainer['params']

We are ready to train our network! As before, we will start a local server to see the training progress of our network (if you server is already running, you should not start another one). Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [ ]:
# Train the network!
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

Make sure you get at least 50% accuracy in this section! If you tried different settings than the ones provided to get 50%, you should modify custom_part1_trainer in student code to return a dictionary with your changed settings. 

## Part 2. Fine-Tuning a Pre-Trained Network

In [ ]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

Training a network from scratch takes a lof of time. Instead of training from scratch, we can take a pre-trained model and fine tune it for our purposes. This is the goal of Part 2--you will train a pre-trained network, and achieve at least 80% accuracy. 

In [ ]:
# training parameters
input_size = (224, 224)
RGB = True
base_lr = 1e-3
weight_decay = 5e-4
momentum = 0.9
backprop_depth = 3

In [ ]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Following block loads a pretrained AlexNet.

In [ ]:
# Create the network model.
model = alexnet(pretrained=True)
print(model)

Now, you modify create_part2_model from student code in order to fine-tune AlexNet. As you can see in the docs (https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py) and in the model printout above, AlexNet has 2 parts: 'features', which constists of conv layers that extract feature maps from the image, and 'classifier' which consists of FC layers that classify the features. We want to replace the last Linear layer in model.classifier. 

In [ ]:
model = sc.create_part2_model(model, num_classes)
if use_GPU:
    model = model.cuda()
print(model)

Next we will create the loss function and the optimizer. Just as with part 1, if you modify any of the setttings to hit the required accuracy, you must modify custom_part2_trainer function to return a dictionary containing your changes. 

In [ ]:
# Set up the trainer. You can modify custom_part2_trainer in
# student_copy.py if you want to try different learning settings.
custom_part2_trainer = sc.custom_part2_trainer(model)

if custom_part2_trainer is None:
    # Create the loss function
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Since we do not want to optimize the whole network, we must extract a list of parameters of interest that will be
    # optimized by the optimizer.
    params_to_optimize = []

    # List of modules in the network
    mods = list(model.features.children()) + list(model.classifier.children())

    # Extract parameters from the last `backprop_depth` modules in the network and collect them in
    # the params_to_optimize list.
    for m in mods[::-1][:backprop_depth]:
        params_to_optimize.extend(list(m.parameters()))

    # Construct the optimizer    
    optimizer = optim.SGD(params=params_to_optimize, lr=base_lr, weight_decay=weight_decay, momentum=momentum)

    # Create a scheduler, currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    params = {'n_epochs': 4, 'batch_size': 10, 'experiment': 'part2'} 
    
else:
    if 'loss_function' in custom_part2_trainer:
        loss_function = custom_part2_trainer['loss_function']
    if 'optimizer' in custom_part2_trainer:
        optimizer = custom_part2_trainer['optimizer']
    if 'lr_scheduler' in custom_part2_trainer:
        lr_scheduler = custom_part2_trainer['lr_scheduler']
    if 'params' in custom_part2_trainer:
        params = custom_part2_trainer['params']

We are ready to fine tune our network! Just like before, we will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 2 on the top left bar where is says Environment (only select Part 2, do not check main or Part 1).

In [ ]:
# Train the network!
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

Expect this code to take around 10 minutes on CPU or 30 seconds on GPU. You should hit 80% accuracy. 